# IMDB Data Preprocessing
**Author:** David S. White <br /> 
**Email:** dswhite2012@gmail.com <br /> 
**Date:** 2020-04-21 <br /> 

### Goal
The goal of this notebook is to prepare the raw data from IMDB into a more suitable data frame for analysis. The data was intially downloaded from https://datasets.imdbws.com/ on 2020-04-21. See https://www.imdb.com/interfaces/ for a full description of each of the files. 

Of this data, I am keen on the following files: 
- *title.ratings.tsv* 
- *title.basics.tsv* 
<br /> 

Therefore, the first step before any data analysis is to merge these two as data-frames and clean to make a more suitable dataset. For this, I will refine my data selection to include films from USA only from the past 49 years (1970-2019).

This will generate *imdb_movies.csv*

In [1]:
# Libraries
import os 
import glob
import pandas as pd 
import numpy as np

In [2]:
# Load Data. May take 1-2 minutes.
path = os.path.join("raw_data")
all_files = glob.glob(path + "/*.tsv")
df_list = []
for filename in all_files:
    df = pd.read_csv(filename, sep='\t')
    print("File Loaded: ", filename)
    df_list.append(df)
      
print("All Files Loaded")
ratings= df_list[0]
basics = df_list[1]

File Loaded:  raw_data/title.ratings.tsv


/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


File Loaded:  raw_data/title.basics.tsv
All Files Loaded


### Clean *ratings*

In [3]:
ratings.head()  

,tconst,averageRating,numVotes
0,tt0000001,5.6,1607
1,tt0000002,6.0,197
2,tt0000003,6.5,1284
3,tt0000004,6.1,121
4,tt0000005,6.1,2050


In [4]:
# make sure values are numeric and drop any possible nan values
ratings["averageRating"] = pd.to_numeric(ratings["averageRating"])
ratings["numVotes"] = pd.to_numeric(ratings["numVotes"])
ratings = ratings[pd.notnull(ratings["numVotes"])]
ratings = ratings[pd.notnull(ratings["averageRating"])]

ratings.describe()

,averageRating,numVotes
count,1.029531e+06,1.029531e+06
mean,6.887102e+00,9.626213e+02
std,1.397518e+00,1.586334e+04
min,1.000000e+00,5.000000e+00
25%,6.100000e+00,9.000000e+00
50%,7.100000e+00,2.000000e+01
75%,7.900000e+00,7.800000e+01
max,1.000000e+01,2.223174e+06


Looks like all values have an ID, averageRating, and numVotes field filled and numeric. Should be good to go.

### Clean *basics*

In [5]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [6]:
basics.titleType.unique()
print(basics.groupby('titleType')['tconst'].nunique())

titleType
movie            548744
short            735110
tvEpisode       4792114
tvMiniSeries      30466
tvMovie          120634
tvSeries         182371
tvShort           12488
tvSpecial         28856
video            262108
videoGame         25337
Name: tconst, dtype: int64


Awesome: This has video games, movie, and tv show data in it. But for now, let's only keep "movie"
Note: genres are off for video games, might be better to look at metacritic

Filter basics by titleType = "movie" and isAdult = 0

In [7]:
basics = basics[basics['titleType'].str.contains("movie")]
basics = basics[basics.isAdult == 0]

drop missing values in "genre", "runtimeMinutes", and "startYear"

In [8]:
basics = basics.replace(r'\\N',  np.nan, regex=True)
basics = basics[pd.notnull(basics["genres"])]
basics = basics[pd.notnull(basics["runtimeMinutes"])]
basics = basics[pd.notnull(basics["startYear"])]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
145,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,20,"Documentary,News,Sport"
571,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Biography,Crime,Drama"
673,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
1173,tt0001184,movie,Don Juan de Serrallonga,Don Juan de Serrallonga,0,1910,NaN,58,"Adventure,Drama"


For this analysis, let's look at movies of the past 45 years (1970-2015). 

In [9]:
# keep startYear between 1970 and 2015
basics["startYear"] = pd.to_numeric(basics["startYear"])
basics["runtimeMinutes"] = pd.to_numeric(basics["runtimeMinutes"])

basics = basics[basics.startYear > 1969]
basics = basics[basics.startYear < 2015]

basics.describe()

,isAdult,startYear,endYear,runtimeMinutes
count,183573.0,183573.000000,0.0,183573.000000
mean,0.0,1999.691785,NaN,90.376820
std,0.0,13.108621,NaN,131.711021
min,0.0,1970.000000,NaN,1.000000
25%,0.0,1990.000000,NaN,76.000000
50%,0.0,2005.000000,NaN,90.000000
75%,0.0,2011.000000,NaN,100.000000
max,0.0,2014.000000,NaN,51420.000000


drop fields no longer needed

In [10]:
basics = basics.drop(["endYear", "originalTitle", "titleType","isAdult"], axis=1)

### Merge the two data frames

Both  tables have **tconst** key that at can be used to join them. Perform an inner join using "tconst* field

In [11]:
# Perform inner join via "tconst". This will remove empty data
movies = merged_inner = pd.merge(left=basics, right=ratings, left_on='tconst', right_on='tconst')
movies.head()

,tconst,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0015724,Dama de noche,1993,102,"Drama,Mystery,Romance",6.2,20
1,tt0016906,Frivolinas,2014,80,"Comedy,Musical",5.6,15
2,tt0031458,El huésped del sevillano,1970,86,Comedy,7.1,8
3,tt0035423,Kate & Leopold,2001,118,"Comedy,Fantasy,Romance",6.4,76740
4,tt0036606,"Another Time, Another Place",1983,118,"Drama,War",6.5,247


In [12]:
movies.describe()

,startYear,runtimeMinutes,averageRating,numVotes
count,127915.000000,127915.000000,127915.000000,1.279150e+05
mean,1998.772716,96.046805,6.144964,5.029301e+03
std,13.161842,153.995931,1.344664,4.055837e+04
min,1970.000000,1.000000,1.000000,5.000000e+00
25%,1989.000000,84.000000,5.300000,1.900000e+01
50%,2003.000000,92.000000,6.300000,7.600000e+01
75%,2010.000000,104.000000,7.100000,4.190000e+02
max,2014.000000,51420.000000,10.000000,2.223174e+06


### Save the data 

In [13]:
movies.to_csv('imdb_movies.csv')

print("Data Saved.")

Data Saved.
